In [1]:
import pandas as pd
import numpy as np
import os

from tqdm import tqdm

import nibabel as nib

import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image


import sys
import torch
from torch import Tensor
import torchvision.transforms as transforms
from torchvision.transforms import Resize, ToTensor
# from models import *

%load_ext autoreload
%autoreload 2

sys.path.insert(0,'/home/roshansk/Covid/CXRData/')
from SegLearner import *

import segmentation_models_pytorch as smp

sys.path.insert(0,'/home/roshansk/Covid/RibFrac/Models/')
from models import *

In [2]:
trainFolder = '/home/roshansk/Covid/Live/ircad-dataset/train/'
sliceFolder = '/home/roshansk/Covid/Live/ircad-dataset/Slices/'

dataFiles = os.listdir(trainFolder)

imgFiles = [x for x in dataFiles if 'orig' in x]

sliceList = sorted(os.listdir(sliceFolder))
imgFiles = [x for x in sliceList if 'image' in x]
labelFiles = [x.replace('image','label') for x in imgFiles]

df = pd.DataFrame({'imgPath':imgFiles,'maskPath':labelFiles})
df.imgPath = df.imgPath.apply(lambda x : os.path.join(sliceFolder, x))
df.maskPath = df.maskPath.apply(lambda x : os.path.join(sliceFolder, x))

print(len(df))

2073


In [3]:
0.8*2073

1658.4

## Modeling

### Setup

In [4]:
device = 'cuda:0'

batchSize = 10
imgSize = 256


model = ResNetDUC(num_classes=1)

# model = smp.Unet('resnet34', encoder_weights='imagenet')
# model = smp.PSPNet('resnet34', encoder_weights='imagenet')



criterion = torch.nn.BCELoss()


optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

model.to(device)


trainTransforms = transforms.Compose([transforms.Resize((imgSize,imgSize)), 
                                      ToTensor()])


# trainTransforms = transforms.Compose([transforms.RandomResizedCrop(size = 256,scale = (0.06,0.5), ratio=(0.75, 1.3)), 
#                                       ToTensor()])

classLoss = nn.CrossEntropyLoss()
reconLoss = nn.MSELoss()

# optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)


trainDataset = SegDataset(df.iloc[0:1650], trainTransforms)
testDataset = SegDataset(df.iloc[1650:len(df)], trainTransforms)

trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=batchSize, 
                                          shuffle=True, num_workers=1)


testLoader = torch.utils.data.DataLoader(testDataset, batch_size=batchSize, 
                                          shuffle=False, num_workers=1)

In [47]:
trainDataset[0][0].shape

torch.Size([3, 256, 256])

In [48]:
trainDataset[0][1].shape

torch.Size([256, 256])

### Training

In [5]:
learner = SegLearner(model, trainLoader, optimizer, criterion, testLoader, device = 'cuda:0')

In [6]:
learner.fit(num_epochs=3, save_best_model=False, save_every_epoch=False, useLogger = False)

  0%|          | 0/43 [00:00<?, ?it/s]

Epoch Time : 0:14:32.442324
Train Loss : 0.6633532495209665  Train IOU : 0.700848400592804


  0%|          | 0/165 [00:00<?, ?it/s]

Test IOU : 0.6023641228675842


  0%|          | 0/43 [00:00<?, ?it/s]

...

Test IOU : 0.6513002514839172


  0%|          | 0/43 [00:00<?, ?it/s]

Epoch Time : 0:14:38.055117
Train Loss : 0.6512809038162232  Train IOU : 0.8215151429176331


100%|██████████| 43/43 [00:19<00:00,  2.23it/s]


Test IOU : 0.6586288809776306


In [7]:
learner.fit(num_epochs=2, save_best_model=False, save_every_epoch=False, useLogger = False)

  0%|          | 0/43 [00:00<?, ?it/s]

Epoch Time : 0:14:33.934375
Train Loss : 0.6504858367370837  Train IOU : 0.8757575750350952


  0%|          | 0/165 [00:00<?, ?it/s]

Test IOU : 0.6794326305389404


  0%|          | 0/43 [00:00<?, ?it/s]

Epoch Time : 0:14:38.391424
Train Loss : 0.6498676997242552  Train IOU : 0.9229090213775635


100%|██████████| 43/43 [00:19<00:00,  2.24it/s]


Test IOU : 0.7063829302787781


### Using NPY files

In [11]:
trainX = '/home/roshansk/Covid/Live/Deep-Learning-liver-segmentation/imgs_train.npy'
testX = '/home/roshansk/Covid/Live/Deep-Learning-liver-segmentation/imgs_test.npy'

trainy = '/home/roshansk/Covid/Live/Deep-Learning-liver-segmentation/masks_train.npy'
testy = '/home/roshansk/Covid/Live/Deep-Learning-liver-segmentation/masks_test.npy'

In [12]:
trainX = np.load(trainX)
testX = np.load(testX)

trainy = np.load(trainy)
testy = np.load(testy)

In [70]:
class SegDataset2(torch.utils.data.Dataset):
    
  def __init__(self, matrix, mask):


    self.matrix = matrix
    self.mask = mask
    self.transforms = transforms


  def __len__(self):

    return self.matrix.shape[0]

  def __getitem__(self, index):
        
        
    img = self.matrix[index,:,:]
    mask = self.mask[index,:,:]
    
    img = Tensor(img).unsqueeze(0).repeat(3,1,1)
    mask = Tensor(mask).unsqueeze(0)
    
#     mask = mask.type(torch.uint8)

    return img, mask

In [71]:
trainDataset = SegDataset2(trainX, trainy)
testDataset = SegDataset2(testX, testy)

In [72]:
trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=batchSize, 
                                          shuffle=True, num_workers=1)


testLoader = torch.utils.data.DataLoader(testDataset, batch_size=batchSize, 
                                          shuffle=False, num_workers=1)

In [53]:
a = trainDataset[0][0]

In [62]:
a.unsqueeze(0).repeat(3,1,1).shape

torch.Size([3, 256, 256])

In [37]:
a[1].dtype

torch.float32

In [73]:
learner = SegLearner(model, trainLoader, optimizer, criterion, testLoader, device = 'cuda:0')

In [74]:
learner.fit(num_epochs=3, save_best_model=False, save_every_epoch=False, useLogger = False)

  0%|          | 0/204 [00:00<?, ?it/s]

Epoch Time : 0:04:23.890350
Train Loss : 0.6960874767426182  Train IOU : 0.309492290019989


  0%|          | 0/272 [00:00<?, ?it/s]

Test IOU : 0.2537856101989746


  0%|          | 0/204 [00:00<?, ?it/s]

...

Test IOU : 0.35437560081481934


  0%|          | 0/204 [00:00<?, ?it/s]

Epoch Time : 0:04:28.421801
Train Loss : 0.6582859603359419  Train IOU : 0.580426812171936


100%|██████████| 204/204 [00:47<00:00,  4.31it/s]


Test IOU : 0.3651917278766632


In [75]:
learner.fit(num_epochs=3, save_best_model=False, save_every_epoch=False, useLogger = False)

  0%|          | 0/204 [00:00<?, ?it/s]

Epoch Time : 0:04:24.470512
Train Loss : 0.6563309336848119  Train IOU : 0.6831493973731995


  0%|          | 0/272 [00:00<?, ?it/s]

Test IOU : 0.4163225293159485


  0%|          | 0/204 [00:00<?, ?it/s]

...

Test IOU : 0.45044249296188354


  0%|          | 0/204 [00:00<?, ?it/s]

Epoch Time : 0:04:28.197490
Train Loss : 0.6542562193730298  Train IOU : 0.7507726550102234


100%|██████████| 204/204 [00:47<00:00,  4.31it/s]

Test IOU : 0.44316619634628296


In [77]:
learner.fit(num_epochs=8, save_best_model=False, save_every_epoch=False, useLogger = False)

  0%|          | 0/204 [00:00<?, ?it/s]

Epoch Time : 0:04:23.707304
Train Loss : 0.6510335700038601  Train IOU : 0.8704193830490112


  0%|          | 0/272 [00:00<?, ?it/s]

Test IOU : 0.45024585723876953


  0%|          | 0/204 [00:00<?, ?it/s]

...

Test IOU : 0.44660767912864685


  0%|          | 0/204 [00:00<?, ?it/s]

Epoch Time : 0:04:28.158554
Train Loss : 0.649992504321477  Train IOU : 0.8989697694778442


100%|██████████| 204/204 [00:47<00:00,  4.31it/s]

Test IOU : 0.44739434123039246


In [ ]:
learner.fit(num_epochs=8, save_best_model=False, save_every_epoch=False, useLogger = False)